In [ ]:
#Install libraries as needed
!pip install gtfs_functions
!pip install gtfs_kit

In [ ]:
#import packages
from gtfs_functions import Feed
import gtfs_kit as gk
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.ops import unary_union
import matplotlib.pyplot as plt

## Import Data
#### Import GTFS 

In [ ]:
#Bring in GTFS data using gtfs_functions

#Change path to local GTFS zip
gtfs_path = r"C:\Users\cathk\GEOG490\google_transit.zip"
feed = Feed(gtfs_path, time_windows=[0, 6, 10, 12, 16, 19, 24])

In [ ]:
#Bring in GTFS .txt files using feed
routes = feed.routes
trips = feed.trips
stops = feed.stops
stop_times = feed.stop_times
shapes = feed.shapes
calendar = feed.calendar

#### Import Census Tract data

In [ ]:
#Change path to local tract shapefile
tracts = gpd.read_file(r"C:\Users\cathk\Documents\490Final\Transportation-Access-\data\shapefiles\tl_2021_17_tract.shp")

#Get county Census tract
county_tracts = tracts[tracts['COUNTYFP'] == '031']

#Reproject county tracts to UTM for the city
county_tracts = county_tracts.to_crs("EPSG:32616")

In [ ]:
# Extract needed columns
county_tracts = county_tracts[['TRACTCE','ALAND','AWATER','geometry']]

# Transit Factors and Indicators

### Factor: Connectivity to the Network
##### Indicators: Bus stop service coverage area, density of bus stops, route coverage, ADA accessibility

#### Indicator: Bus stop service coverage area
Bus stop service coverage area is determined by finding the sum of the service coverage areas of bus stops in a Census tract divided by the total area of the Census tract.

 - Service coverage areas are 400m radius buffers from each bus stop. Here overlapping areas are counted one time.


In [ ]:
# Bring in the bus stops
stops = feed.stops

#Reproject the bus stops to UTM
stops_reproject = stops.to_crs("EPSG:32616")

#buffer stops 
buffer_distance = 400 #meters
stops_buffered = stops_reproject.buffer(buffer_distance)

#Convert buffered stops geoseries to a geodataframe
stops_buffered_gdf = gpd.GeoDataFrame(geometry=stops_buffered)

In [ ]:
# Calculate the areas
county_tracts['area'] = county_tracts.geometry.area

In [ ]:
# Merge all the polygons into a single polygon
merged_polygon = unary_union(stops_buffered_gdf.geometry)

# Create a new GeoDataFrame with the merged polygon
merged_gdf = gpd.GeoDataFrame(geometry=[merged_polygon])

# Reset the index 
merged_gdf.reset_index(drop=True, inplace=True)


In [ ]:
# Create an empty list to store the results
area_covered_list = []

# Iterate over each row in gdf2
for _, row in county_tracts.iterrows():
    # Calculate the intersection between the row geometry of gdf2 and the geometries in gdf1
    intersection = merged_gdf.intersection(row.geometry)
    
    # Calculate the total area of the intersection
    total_intersection_area = intersection.area.sum()
    
    # Append the total intersection area to the list
    area_covered_list.append(total_intersection_area)

# Add the area_covered_list as a new column to gdf2
county_tracts['area_covered'] = area_covered_list



In [ ]:
# Get the service coverage area by dividing the area_covered by the total area of the tract
county_tracts['service_coverage']=county_tracts['area_covered']/county_tracts['area']

#### Indicator: Density of bus stops
The density of bus stops is determined by dividing the number of bus stops in a census tract by the area of the census tract

In [ ]:
# Ensure that the TRACTCE column is the same type as in the GTFS data
county_tracts['TRACTCE'] = county_tracts['TRACTCE'].astype(int)

In [ ]:
# Get the number of bus stops in a census tract
stops_county = stops_reproject.sjoin(county_tracts, how="left")

# Calculate the area
stops_county['area'] = stops_county['geometry'].area

# Get the unique counties in the bus stops gdf and count the bus stops in each.
unique, counts = np.unique(stops_county['TRACTCE'], return_counts=True)
dict(zip(unique, counts))

# Turn the unique counts dictionary into a gdf
busstopsincounty = gpd.GeoDataFrame(list(zip(unique, counts)), 
                       columns=['TRACTCE', 'bus_stops'])

# Merge the number of bus stops gdf with the county_tracts gdf
county_tracts = county_tracts.merge(busstopsincounty, on='TRACTCE', how='left')

# Calculate the density of bus stops by dividing the number of bus stops by the area of each census tract
county_tracts['density_bstops'] = county_tracts['bus_stops']/county_tracts['area']

#### Indicator: Route coverage
Route coverage quantifies the distribution and density of bus routes throughout the street network of the Census tract. This metric is calculated by taking the sum of the lengths of all routes in a census tract and dividing this by the sum of the length of the street network throughout the tract.

In [ ]:
# Load the GTFS feed into a Feed object
feed = gk.read_feed(gtfs_path,dist_units='mi')

In [ ]:
# Use the geometrize_routes function from gtfs_kits to get the geometry of the routes
routesgeom = gk.routes.geometrize_routes(feed)

In [ ]:
# Set the crs of the geometrized routes based on the stored geometry values
routesgeom.crs = "EPSG:4326" 

#Reproject the routes to UTM fo the city
routes_reproject = routesgeom.to_crs("EPSG:32616")

In [ ]:
# Get bus routes (type 3)
routes_reproject = routes_reproject[routes_reproject['route_type'] == 3]

In [ ]:
# Sum length of bus routes in each tract

# Perform a spatial join to get the lines that intersect with the polygons
lines_within_polygons = gpd.overlay(routes_reproject, county_tracts, how='intersection')

# Calculate the length of the resulting LineStrings
lines_within_polygons['length'] = lines_within_polygons.geometry.length

# Group by 'TRACTCE' to handle unique polygons and calculate the sum of the lengths for each
total_length_within_polygons = lines_within_polygons.groupby('TRACTCE')['length'].sum().reset_index()

# Rename the 'length' column to 'routesum'
total_length_within_polygons.rename(columns={'length': 'routesum'}, inplace=True)

In [ ]:
# Merge the routesum into the tracts gdf
county_tracts = county_tracts.merge(total_length_within_polygons, on='TRACTCE', how='left')

In [ ]:
'''
Takes too long to run 

import osmnx as ox
import geopandas as gpd

# Define  ity or region of interest
place_name = "Chicago, Illinois, USA"

network_lengths = []
for geometry in county_tracts['geometry']:

    
    # Retrieve the street network within the boundary of the Census Tract
    G = ox.graph_from_polygon(geometry, network_type='all')
    
     # Calculate the basic statistics of the street network within the Census Tract
    stats = ox.stats.basic_stats(G)
    
    # Extract the total length of the street network within the Census Tract
    total_length = stats['edge_length_total']
    
    network_lengths.append(total_length)

    print("count")
county_tracts['net_len'] = network_lengths

countforarea['route_covg'] = countforarea['routesum']/countforarea['net_len']
'''

#### Indicator: ADA accessibility
Here the number of stops with ADA ramps is used as a proxy for system-wide accesibility. The number of stops with ADA ramps is used here.

In [ ]:
# Get the number of bus stops in a census tract
# Spatially join stops and census tracts
stops_county = gpd.sjoin(stops_reproject, county_tracts, how="left")

# Convert 'wheelchair_boarding' column to integer
stops_county['wheelchair_boarding'] = stops_county['wheelchair_boarding'].astype(int)

# Filter stops with wheelchair value of 1
wheelchair_stops = stops_county[stops_county['wheelchair_boarding'] == 1]

# Count the number of wheelchair stops within each census tract
wheelchair_stops_count = wheelchair_stops.groupby('TRACTCE').size().reset_index(name='wheelchair_stops')

# Ensure all census tracts are included in the result and replace null values with 0
all_tracts = county_tracts['TRACTCE']
wheelchair_stops_count = all_tracts.merge(wheelchair_stops_count, on='TRACTCE', how='left').fillna(0)

# Merge the counts with the county_tracts GeoDataFrame
county_tractsalt = county_tracts.merge(wheelchair_stops_count, on='TRACTCE', how='left')

## Create scores from transit supply indicators

In [ ]:
from scipy.stats import zscore

In [ ]:
# Remove tracts with null values
county_dropped = county_tracts.dropna()

In [ ]:
# Standarize Data
columns_to_standardize = ['service_coverage','density_bstops','routesum','wheelchair_stops']

# Calculate z-scores for the specified columns
county_dropped[columns_to_standardize] = county_dropped[columns_to_standardize].apply(zscore)

In [ ]:
# Sum the z-scores across the specified columns
county_dropped['transit_supply_score'] = county_dropped[columns_to_standardize].sum(axis=1)

In [ ]:
# Group the transit supply scores into quartiles
county_dropped['quartile'] = pd.qcut(county_dropped['transit_supply_score'], 4, labels=['Low', 'Moderate-Low', 'Moderate-High', 'High'])

In [ ]:
# Make the quartile df into a gdf
gdf = gpd.GeoDataFrame(county_dropped, geometry=county_dropped['geometry'])

In [ ]:
gdf

In [ ]:
# Convert the categorical column to a string
gdf['quartile'] = gdf['quartile'].astype(str)


gdf.to_file(r"C:\Users\cathk\GEOG490\chicago_transit_supply.shp")


In [ ]:
# Plot census tracts by transit supply quartile

# Ensure 'quartile' column exists and contains the expected values
print(gdf['quartile'].value_counts())

# Define a color map for the quartiles
quartile_colors = {
    'Low': 'blue',
    'Moderate-Low': 'cyan',
    'Moderate-High': 'orange',
    'High': 'red'
}

# Plot the GeoDataFrame with the specified colors for quartiles
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

gdf.plot(column='quartile', 
         ax=ax, 
         legend=True,
         legend_kwds={'title': "Transit Supply Quartiles"},
         color=[quartile_colors[q] for q in gdf['quartile']])

# Add a title and axes labels
ax.set_title('Transit Supply by Census Tract')
ax.set_axis_off()

# Show the plot
plt.show()